In [16]:
!pip install pyspark
# Spark SQL
!pip install pyspark[sql]
# pandas API on Spark
!pip install pyspark[pandas_on_spark] plotly  # to plot your data, you can install plotly together.
# Spark Connect
!pip install pyspark[connect]


In [17]:
!pip install findspark

In [32]:
import findspark
from pyspark.sql.types import *
from pyspark.sql.functions import *

findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark

In [19]:
df = spark.read.csv('dataset.csv')

df = df.withColumnRenamed("_c0", "Transaction_unique_identifier") \
        .withColumnRenamed("_c1", "price") \
        .withColumnRenamed("_c2", "Date_of_Transfer") \
        .withColumnRenamed("_c3", "postcode") \
        .withColumnRenamed("_c4", "Property_Type") \
        .withColumnRenamed("_c5", "Old_New") \
        .withColumnRenamed("_c6", "Duration") \
        .withColumnRenamed("_c7", "PAON") \
        .withColumnRenamed("_c8", "SAON") \
        .withColumnRenamed("_c9", "Street") \
        .withColumnRenamed("_c10", "Locality") \
        .withColumnRenamed("_c11", "Town_City") \
        .withColumnRenamed("_c12", "District") \
        .withColumnRenamed("_c13", "County") \
        .withColumnRenamed("_c14", "PPDCategory_Type") \
        .withColumnRenamed("_c15", "idk")
df.head()

Row(Transaction_unique_identifier='{F887F88E-7D15-4415-804E-52EAC2F10958}', price='70000', Date_of_Transfer='1995-07-07 00:00', postcode='MK15 9HP', Property_Type='D', Old_New='N', Duration='F', PAON='31', SAON=None, Street='ALDRICH DRIVE', Locality='WILLEN', Town_City='MILTON KEYNES', District='MILTON KEYNES', County='MILTON KEYNES', PPDCategory_Type='A', idk='A')

In [20]:
df.columns

['Transaction_unique_identifier',
 'price',
 'Date_of_Transfer',
 'postcode',
 'Property_Type',
 'Old_New',
 'Duration',
 'PAON',
 'SAON',
 'Street',
 'Locality',
 'Town_City',
 'District',
 'County',
 'PPDCategory_Type',
 'idk']

1.Найти среднюю цену среди всех транзакций, сгруппированных по типу собственности и длительности владения:

In [26]:
df.groupBy("Property_Type", "Duration").agg(avg("price").alias("Average_Price")).show()

+-------------+--------+------------------+
|Property_Type|Duration|     Average_Price|
+-------------+--------+------------------+
|            S|       F| 61570.41604239112|
|            S|       U|65331.666666666664|
|            T|       F|52674.805845672694|
|            D|       F|106729.03828337233|
|            F|       F| 61837.73195412638|
|            D|       U|           96990.0|
|            F|       U|           61879.0|
|            S|       L|  51257.6982875169|
|            D|       L|  85242.8239777526|
|            F|       L| 58614.61169561398|
|            T|       L| 41629.88612244898|
|            O|       L|           80500.0|
|            O|       F|         1183323.2|
|            T|       U|           26000.0|
+-------------+--------+------------------+



Посчитать общую сумму продаж по каждому городу:

In [27]:
df.groupBy("Town_City").agg(sum("price").alias("Total_Sales")).show()

+--------------------+------------+
|           Town_City| Total_Sales|
+--------------------+------------+
|        HIGH WYCOMBE|2.61111892E8|
| ST. LEONARDS-ON-SEA| 5.6266978E7|
|             NEWQUAY| 3.6387485E7|
|  BERWICK-UPON-TWEED| 1.4048442E7|
|            GOODWICK|    850150.0|
|           ROTHERHAM|1.41388844E8|
|           LLANGEFNI|   3957991.0|
|            STAMFORD| 5.4630079E7|
|         CHOPPINGTON|   6101166.0|
|   LEE-ON-THE-SOLENT| 1.5707105E7|
|           FERRYSIDE|    801980.0|
|         LOSTWITHIEL|   5195350.0|
|             FAREHAM|1.78979447E8|
|             CHORLEY| 7.8830795E7|
|             HARWICH| 2.1678329E7|
|              DUNMOW| 5.0744103E7|
|             ROYSTON| 7.2737277E7|
|              HESSLE| 1.8661145E7|
|       RICKMANSWORTH|1.17080265E8|
|LLANFAIRPWLLGWYNGYLL|   5356695.0|
+--------------------+------------+
only showing top 20 rows



3. Найти самую дорогую транзакцию для каждого типа собственности:

In [28]:
df.groupBy("District").agg(avg("price").alias("Average_Price")).show()

+------------------+------------------+
|          District|     Average_Price|
+------------------+------------------+
|      NORTH DORSET| 77028.29382093316|
|CREWE AND NANTWICH| 58272.65777993214|
|BERWICK-UPON-TWEED| 53042.57534246575|
|         ROTHERHAM| 46590.89839704069|
|         ABERCONWY| 53561.19313850063|
|          SANDWELL| 44140.04236262406|
|           FAREHAM| 75453.08991008991|
|           RUTLAND| 82580.70605612999|
|           CHORLEY| 55090.44198250729|
|     MILTON KEYNES| 62589.67295597484|
|  SOUTH DERBYSHIRE| 57211.54988262911|
|    MERTHYR TYDFIL| 35512.18965517241|
|              HART|104458.23519163764|
|        WINCHESTER|110275.49613402062|
|        BRIDGNORTH| 76207.89819724284|
|       LLIW VALLEY|43399.583662714096|
|           CARRICK| 64213.11045071818|
|       MID SUFFOLK| 66462.42877767084|
|    NORTH CORNWALL| 59466.36515809051|
|   NORTH WILTSHIRE|  75156.2750798722|
+------------------+------------------+
only showing top 20 rows



4. Посчитать количество транзакций, где цена превышает среднюю цену по всем транзакциям:

In [29]:
average_price = df.select(avg("price")).first()[0]
df.filter(col("price") > average_price).count()

364889

5. Найти среднюю цену по каждому типу собственности:

In [34]:
df.groupBy("Property_Type").agg(avg("price").alias("Average_Price")).show()

+-------------+-----------------+
|Property_Type|    Average_Price|
+-------------+-----------------+
|            F|58704.03892185708|
|            T|51648.06572691853|
|            D|105939.9987445452|
|            O|693179.5555555555|
|            S| 60734.7141465879|
+-------------+-----------------+

